<a href="https://colab.research.google.com/github/DAbbottPersonal/PM_china/blob/main/PM_china.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multiple Linear Regression

## Importing the libraries

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Importing the dataset

There are several PM reading for each dataset. Store all of them as a dictionary of numpy arrays for now.

In [6]:
dataset = pd.read_csv('/content/BeijingPM20100101_20151231.csv')
#X = dataset.iloc[:, :-1].values
features = ['No', 
            'year', 
            'month', 
            'day', 
            'hour', 
            'season', 
            'DEWP', 
            'HUMI',
            'PRES', 
            'TEMP', 
            'cbwd', 
            'Iws', 
            'precipitation', 
            'Iprec']
X = dataset.loc[:, features].values
y = {}
for i in dataset.columns:
  if 'PM_' in i:
    y[i] = dataset.loc[:, i].values

In [7]:
print(type(X))
print(X[0:4,:])
print(y)

<class 'numpy.ndarray'>
[[1 2010 1 1 0 4 -21.0 43.0 1021.0 -11.0 'NW' 1.79 0.0 0.0]
 [2 2010 1 1 1 4 -21.0 47.0 1020.0 -12.0 'NW' 4.92 0.0 0.0]
 [3 2010 1 1 2 4 -21.0 43.0 1019.0 -11.0 'NW' 6.71 0.0 0.0]
 [4 2010 1 1 3 4 -21.0 55.0 1019.0 -14.0 'NW' 9.84 0.0 0.0]]
{'PM_Dongsi': array([ nan,  nan,  nan, ..., 171., 204.,  nan]), 'PM_Dongsihuan': array([ nan,  nan,  nan, ..., 231., 242.,  nan]), 'PM_Nongzhanguan': array([ nan,  nan,  nan, ..., 196., 221.,  nan]), 'PM_US Post': array([ nan,  nan,  nan, ..., 203., 212., 235.])}


## Data Preprocessing


### Imputing

I need to remove the NA values and handle large periods of time without any PM data from the other chinese locations. The general strategy is:


1.   Impute the NA values of the US Post. These values are for the most part ubiquitous in the dataset with execption for of a few days.
2.   Do not impute the other locations. If they are available, average them together (with the US post and other chinese locations).
3.   Use the average as the final PM value.

I am also considering MTR (multi-target regression) but for now just use step 1






1: Impute the US_post values

In [8]:
from sklearn.impute import SimpleImputer
print(y['PM_US Post'])
print(len(y['PM_US Post']))
print(np.count_nonzero(np.isnan(y['PM_US Post'].astype(np.float))))
y_prep = np.vstack(y['PM_US Post'])
print(y_prep)
US_post_imputer = SimpleImputer(missing_values=np.nan, strategy='median')
US_post_imputer.fit(y_prep)
y['PM_US Post'] = US_post_imputer.transform(y_prep)
print(y['PM_US Post'])
print(len(y['PM_US Post']))
print(np.count_nonzero(np.isnan(y['PM_US Post'].astype(np.float))))

[ nan  nan  nan ... 203. 212. 235.]
52584
2197
[[ nan]
 [ nan]
 [ nan]
 ...
 [203.]
 [212.]
 [235.]]
[[ 69.]
 [ 69.]
 [ 69.]
 ...
 [203.]
 [212.]
 [235.]]
52584
0


2: Average locations (TO DO)

3: Store Average (TO DO)

There are some values that need imputing in the feature set.
*    Precipitiation (Precipitation and lprec): Use median as mean could produce odd or tiny values of precipitaion.
*    Wind direction (cbwd): Use local average of neighbors (p/m 2 rows) since we don't expect the wind direction to change suddenly (or at least that is my intuition!). This is categorical and will have to be encoded before manipulation.
*    Other values with NAA (DEWP, HUMI, PRES, TEMP, LWS): Use local average of neighbors for this as well but unlike wind direction this is numerical.

Precipitation:

In [9]:
#print(X[42898:42903,-2:])
print("Before and after imputing: ")
print(np.count_nonzero(np.isnan(X[:,-2:].astype(np.float))))
prec_imputer = SimpleImputer(missing_values=np.nan, strategy='median')
prec_imputer.fit(X[:,-2:])
X[:,-2:] = prec_imputer.transform(X[:,-2:])
print(np.count_nonzero(np.isnan(X[:,-2:].astype(np.float))))

Before and after imputing: 
968
0


Other features (not wind direction):

In [10]:
from pandas.core.frame import DataFrame
print(X[45920:45925,:])
print(np.count_nonzero(np.isnan(X[:,[-8,-7,-6,-5,-3]].astype(np.float))))


for col in [-8,-7,-6,-5,-3]:
  df = DataFrame(data = X[:,col])
  df = df.iloc[:, -1].astype(float).interpolate(method='linear')
  X[:,col] = df


print(X[45920:45925,:])
print(np.count_nonzero(np.isnan(X[:,[-8,-7,-6,-5,-3]].astype(np.float))))

[[45921 2015 3 29 8 1 3.0 50.0 1018.0 13.0 'cv' 0.89 0.0 0.0]
 [45922 2015 3 29 9 1 4.0 47.0 1018.0 15.0 'SE' 3.13 0.0 0.0]
 [45923 2015 3 29 10 1 nan nan nan nan nan nan 0.0 0.0]
 [45924 2015 3 29 11 1 4.0 41.0 1018.0 17.0 'SE' 3.13 0.0 0.0]
 [45925 2015 3 29 12 1 4.0 34.0 1015.0 20.0 'SE' 8.05 0.0 0.0]]
693
[[45921 2015 3 29 8 1 3.0 50.0 1018.0 13.0 'cv' 0.89 0.0 0.0]
 [45922 2015 3 29 9 1 4.0 47.0 1018.0 15.0 'SE' 3.13 0.0 0.0]
 [45923 2015 3 29 10 1 4.0 44.0 1018.0 16.0 nan 3.13 0.0 0.0]
 [45924 2015 3 29 11 1 4.0 41.0 1018.0 17.0 'SE' 3.13 0.0 0.0]
 [45925 2015 3 29 12 1 4.0 34.0 1015.0 20.0 'SE' 8.05 0.0 0.0]]
0


Wind direction:

In [11]:
winds = X[:,-4]
new_cols = np.zeros((len(winds),5))
winds = np.vstack(winds)
new_cols = np.concatenate((winds, new_cols),1)


new_cols[new_cols[:,0] == 'NE'] = ['NE', 1., 0., 0., 0., 0.]
new_cols[new_cols[:,0] == 'NW'] = ['NW', 0., 1., 0., 0., 0.]
new_cols[new_cols[:,0] == 'SW'] = ['SW', 0., 0., 1., 0., 0.]
new_cols[new_cols[:,0] == 'SE'] = ['SE', 0., 0., 0., 1., 0.]
new_cols[new_cols[:,0] == 'cv'] = ['cv', 0., 0., 0., 0., 1.]
new_cols[new_cols[:,0] == 'nan'] = ['nan', np.NaN, np.NaN, np.NaN, np.NaN, np.NaN]


print(np.count_nonzero(np.isnan(new_cols[:,1].astype(np.float))))


df = DataFrame(data = new_cols)
df = df.iloc[:, 1:].astype(float).fillna(method='bfill')
final = df.values
print(np.count_nonzero(np.isnan(final[:,1].astype(np.float))))

print(X[45920:45925,:])
X = np.concatenate((X,final),1)
print(X[21670:21680,:])

5
0
[[45921 2015 3 29 8 1 3.0 50.0 1018.0 13.0 'cv' 0.89 0.0 0.0]
 [45922 2015 3 29 9 1 4.0 47.0 1018.0 15.0 'SE' 3.13 0.0 0.0]
 [45923 2015 3 29 10 1 4.0 44.0 1018.0 16.0 nan 3.13 0.0 0.0]
 [45924 2015 3 29 11 1 4.0 41.0 1018.0 17.0 'SE' 3.13 0.0 0.0]
 [45925 2015 3 29 12 1 4.0 34.0 1015.0 20.0 'SE' 8.05 0.0 0.0]]
[[21671 2012 6 21 22 2 18.0 73.0 1005.0 23.0 'SE' 21.01 2.6 2.7 0.0 0.0
  0.0 1.0 0.0]
 [21672 2012 6 21 23 2 19.0 88.0 1004.0 21.0 'SE' 25.03 1.1 3.8 0.0 0.0
  0.0 1.0 0.0]
 [21673 2012 6 22 0 2 19.0 93.0 1004.0 20.0 'SE' 28.16 0.1 3.9 0.0 0.0
  0.0 1.0 0.0]
 [21674 2012 6 22 1 2 19.0 93.0 1004.0 20.0 'SE' 33.97 0.5 4.4 0.0 0.0
  0.0 1.0 0.0]
 [21675 2012 6 22 2 2 19.0 93.0 1004.0 20.0 'cv' 0.89 0.1 4.5 0.0 0.0 0.0
  0.0 1.0]
 [21676 2012 6 22 3 2 20.0 100.0 1004.0 20.0 'NW' 1.79 0.0 0.0 0.0 1.0
  0.0 0.0 0.0]
 [21677 2012 6 22 4 2 19.0 93.0 1004.0 20.0 'NW' 4.92 0.2 0.2 0.0 1.0 0.0
  0.0 0.0]
 [21678 2012 6 22 5 2 19.0 93.0 1004.0 20.0 'SE' 0.89 0.0 0.0 0.0 0.0 0.0
  1.0 0

Last, I will clean out the variables that may not be helpful like the string value of the wind direction and event number (since this will not be useful in prediction).

In [20]:
X_final = X[:,[1,2,3,4,5,6,7,8,9,11,12,13,14,15,16,17,18]]
features_final = ['year',
                  'month',
                  'day',
                  'hour',
                  'season',
                  'DEWP',
                  'HUMI',
                  'PRES',
                  'TEMP',
                  'lws',
                  'precipitation',
                  'lprec',
                  'NE',
                  'NW',
                  'SW',
                  'SE',
                  'cv']
y_final = np.vstack(y['PM_US Post'])

## Splitting the dataset into the Training set and Test set

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y['PM_US Post'], test_size = 0.2, random_state = 0)

## Training the Multiple Linear Regression model on the Training set

In [ ]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

### Predicting the Test set results

In [ ]:
y_pred = regressor.predict(X_test)
np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[ 73.92  35.  ]
 [114.4   55.  ]
 [ 80.35  11.  ]
 ...
 [120.26  81.  ]
 [ 94.63  77.  ]
 [ 56.53  57.  ]]


### Evaluating the Model Performance

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.2839919035992807

## Training Polynomial Regression

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly_reg = PolynomialFeatures(degree = 4)
X_poly = poly_reg.fit_transform(X_train)
regressor = LinearRegression()
regressor.fit(X_poly, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

### Predicting the Test set results

In [ ]:
y_pred = regressor.predict(poly_reg.transform(X_test))
np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[ 11.78  35.  ]
 [ 55.39  55.  ]
 [ 40.96  11.  ]
 ...
 [170.77  81.  ]
 [171.61  77.  ]
 [ 90.88  57.  ]]


### Evaluating the Model Performance

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.2261452363395572

## Training SVR

First I need to feature scale

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_y = StandardScaler()
X_train_tran = sc_X.fit_transform(X_train)
y_train_tran = sc_y.fit_transform(y_train)

Now Train

In [ ]:
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf')
regressor.fit(X_train_tran, y_train_tran)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

### Predicting the test set results

In [ ]:
y_pred = sc_y.inverse_transform(regressor.predict(sc_X.transform(X_test)))
np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[ 45.93  35.  ]
 [ 53.58  55.  ]
 [ 23.57  11.  ]
 ...
 [112.26  81.  ]
 [132.13  77.  ]
 [ 79.62  57.  ]]


Evaluating the model performance

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.5271147748923304

## Training Random Forest Regression

In [15]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
regressor.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

### Predicting the test set results

In [16]:
y_pred = regressor.predict(X_test)
np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[49.45 35.  ]
 [81.74 55.  ]
 [10.51 11.  ]
 ...
 [99.2  81.  ]
 [89.07 77.  ]
 [72.68 57.  ]]


### Evaluating the model performance

In [17]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.8292150375914632

## Using the Model

I find that a random forest predicts the PM levels with the best R^2 score. This can then be used in the future to predict the PM levels based on the variety of features. What is left is analysizng the features some more to define a baseline as to what qualifies a "high PM2.5" day.

### Analyzing the impact of the variables

In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 4.3MB/s 


In [21]:
import eli5
from eli5.sklearn import PermutationImportance
#perm = PermutationImportance(regressor, random_state=0).fit(y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1))
perm = PermutationImportance(regressor, random_state=0).fit(X_test, y_test.reshape(len(y_test),1))
print(eli5.format_as_text(eli5.explain_weights(perm, feature_names=features_final)))

Explained as: feature importances

Feature importances, computed as a decrease in score when feature
values are permuted (i.e. become noise). This is also known as 
permutation importance.

If feature importances are computed on the same data as used for training, 
they don't reflect importance of features for generalization. Use a held-out
dataset if you want generalization feature importances.

1.4283 ± 0.0328  month
0.7075 ± 0.0274  season
0.6012 ± 0.0108  DEWP
0.3918 ± 0.0120  HUMI
0.3312 ± 0.0128  day
0.2172 ± 0.0093  PRES
0.1807 ± 0.0048  year
0.1393 ± 0.0112  lws
0.0887 ± 0.0057  NW
0.0524 ± 0.0067  hour
0.0470 ± 0.0030  TEMP
0.0157 ± 0.0026  NE
0.0042 ± 0.0005  SE
0.0021 ± 0.0003  lprec
0.0009 ± 0.0003  cv
0.0007 ± 0.0002  precipitation
     0 ± 0.0000  SW


Some interesting take away is that it seems to matter a lot what time of the year it is. Also, the humidity, dew point, and pressure are very important.

## Determining a High PM2.5 day

The definition for a high PM2.5 day is not standardized globally, but well respected standards in the United States and Europe are defined. A daily air density of PM2.5 particles above the thresholds of 35 micrograms/m^3 and 25 micrograms/m^3 represent the standard of high PM2.5 days for the US EPA and European Union respectively. For this study, we will choose the more conservative threshold of 35 micrograms/m^3 to define a high PM2.5 day but our analysis and modelling up until this point is entirely independent of this choice.


In [35]:
mu_thres = 35
sample_days = [['year',
                  'month',
                  'day',
                  'hour',
                  'season',
                  'DEWP',
                  'HUMI',
                  'PRES',
                  'TEMP',
                  'lws',
                  'precipitation',
                  'lprec',
                  'NE',
                  'NW',
                  'SW',
                  'SE',
                  'cv']]
sample_days.append([2021,
                    7,
                    15,
                    12,
                    2,
                    26,
                    79,
                    1020,
                    28,
                    2,
                    0.,
                    0.01,
                    1,
                    0,
                    0,
                    1,
                    0])
print(np.array(sample_days[1]).reshape(1, -1))
pred_PM = regressor.predict(np.array(sample_days[1]).reshape(1, -1))

if pred_PM > mu_thres:
  print("I predict this day is a HIGH PM2.5 day,")
else:
  print("I predict this day is a LOW PM2.5 day,")
print("with a PM2.5 value of " + str(pred_PM[-1]))

[[2.02e+03 7.00e+00 1.50e+01 1.20e+01 2.00e+00 2.60e+01 7.90e+01 1.02e+03
  2.80e+01 2.00e+00 0.00e+00 1.00e-02 1.00e+00 0.00e+00 0.00e+00 1.00e+00
  0.00e+00]]
I predict this day is a HIGH PM2.5 day,
with a PM2.5 value of 86.92
